# Water Quality CSV Transects joined by TIME/DATE 

In [1]:
import os
import pandas
from wq_gps_match import timestamp_match

In [2]:
timestamp_match.TimestampFromDateTime([2013, 4, 4], '08:18:47am')

datetime.datetime(2013, 4, 4, 8, 18, 47)

In [3]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# example shp
shp = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_GPS", "040413_PosnPnt.shp")
# example water quality
wq_file = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_WQ", "Arc_040413_wqt_cc.csv")

print(shp)
print(wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Arc_040413_GPS\040413_PosnPnt.shp
C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Arc_040413_WQ\Arc_040413_wqt_cc.csv


Import CSV file from the MiniSonde as Pandas Dataframe

In [4]:
water_quality = timestamp_match.wq_from_csv(wq_file)
print(water_quality.head())

            Date_Time   Temp    pH SpCond   Sal DEP25  PAR RPAR TurbSC   CHL  \
1 2013-04-04 13:01:03  17.40  7.84    520  0.26  0.39  573  417   13.4  5.34   
2 2013-04-04 13:01:04  17.40  7.84    520  0.26  0.35  573  417   13.4  5.34   
3 2013-04-04 13:01:05  17.40  7.88    520  0.26  0.35  576  419   13.5  5.29   
4 2013-04-04 13:01:06  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   
5 2013-04-04 13:01:07  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   

     CHL.1  
1  0.05078  
2  0.05078  
3  0.05095  
4  0.05073  
5  0.05073  


In [5]:
shp_df = timestamp_match.shp2dataframe(shp)

print(shp_df.head())

       GPS_Date    GPS_Time                               XY
0  [2013, 4, 4]  08:18:47am  [-121.675981849, 38.2332247674]
1  [2013, 4, 4]  08:18:48am  [-121.675986001, 38.2332196115]
2  [2013, 4, 4]  08:18:49am   [-121.675990421, 38.233214315]
3  [2013, 4, 4]  08:18:50am   [-121.675994058, 38.233209039]
4  [2013, 4, 4]  08:18:51am  [-121.675997428, 38.2332039378]


In [6]:
# combine the GPS_date and GPS time fields to match a timestamp to use as join field
timestamp_match.addCombinedDateTime(shp_df, "GPS_Date", "GPS_Time")
print(shp_df.head())

       GPS_Date    GPS_Time                               XY  \
0  [2013, 4, 4]  08:18:47am  [-121.675981849, 38.2332247674]   
1  [2013, 4, 4]  08:18:48am  [-121.675986001, 38.2332196115]   
2  [2013, 4, 4]  08:18:49am   [-121.675990421, 38.233214315]   
3  [2013, 4, 4]  08:18:50am   [-121.675994058, 38.233209039]   
4  [2013, 4, 4]  08:18:51am  [-121.675997428, 38.2332039378]   

            Date_Time  
0 2013-04-04 08:18:47  
1 2013-04-04 08:18:48  
2 2013-04-04 08:18:49  
3 2013-04-04 08:18:50  
4 2013-04-04 08:18:51  


In [7]:
merged = timestamp_match.JoinByTimeStamp(water_quality, shp_df)
#print(merged)

matches = merged[0]

print('Dimensions - origninal WQ:  {},  Matches: {}'.format( water_quality.shape, matches.shape))

print('Number of rows different: {}'.format(water_quality.shape[0] - matches.shape[0]))

percent_match = timestamp_match.JoinMatchPercent(water_quality, matches)

print('Percent Match: {} %'.format(percent_match))

Dimensions - origninal WQ:  (977, 11),  Matches: (934, 14)
Number of rows different: 43
Percent Match: 95.5987717503 %


In [8]:
# add sources to the data frame
timestamp_match.addsourcefield(matches, "WQ_SOURCE", wq_file)
timestamp_match.addsourcefield(matches, "GPS_SOURCE", shp)
print(matches.head())

            Date_Time   Temp    pH SpCond   Sal DEP25  PAR RPAR TurbSC   CHL  \
2 2013-04-04 13:01:05  17.40  7.88    520  0.26  0.35  576  419   13.5  5.29   
3 2013-04-04 13:01:06  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   
4 2013-04-04 13:01:07  17.41  7.85    521  0.26  0.39  576  417   13.5  5.31   
5 2013-04-04 13:01:08  17.40  7.84    521  0.26  0.37  576  415   13.4  5.28   
6 2013-04-04 13:01:09  17.40  7.84    521  0.26  0.37  576  415   13.4  5.28   

     CHL.1      GPS_Date    GPS_Time                               XY  \
2  0.05095  [2013, 4, 4]  01:01:05pm  [-121.799047195, 38.2594938899]   
3  0.05073  [2013, 4, 4]  01:01:06pm  [-121.799022435, 38.2594940117]   
4  0.05073  [2013, 4, 4]  01:01:07pm  [-121.798997863, 38.2594949788]   
5  0.05064  [2013, 4, 4]  01:01:08pm  [-121.798972489, 38.2594960077]   
6  0.05064  [2013, 4, 4]  01:01:09pm  [-121.798945956, 38.2594968398]   

               WQ_SOURCE          GPS_SOURCE  
2  Arc_040413_wqt_cc.csv  040413_

# ZOOP CHL W points

In [9]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))
# shapefile of the Chl sampling locations
station_shp = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_GPS", "040413_ZoopChlW.shp")
# water quality vertical profile for a Chl sampling point
station_wq_file = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_WQ", "Arc_040413_wqp_ca1.csv")

print(station_shp)
print(station_wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Arc_040413_GPS\040413_ZoopChlW.shp
C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Arc_040413_WQ\Arc_040413_wqp_ca1.csv


In [10]:
# let's process that water quality at the stationary point
station_water_quality = timestamp_match.wq_from_csv(station_wq_file)
print(station_water_quality.head())


            Date_Time   Temp    pH SpCond   Sal   DO%    DO DEP25   PAR RPAR  \
1 2013-04-04 18:21:17  17.67  7.72    659  0.34  61.8  4.88  0.33  1138  410   
2 2013-04-04 18:21:21  55.00  0.00      0  0.00   0.0  0.00  0.00     0    0   
3 2013-04-04 18:21:25  55.00  0.00      0  0.00   0.0  0.00  0.00     0    0   
4 2013-04-04 18:21:26  55.00  0.00      0  0.00   0.0  0.00  0.00     0    0   
5 2013-04-04 18:21:27  55.00  0.00      0  0.00   0.0  0.00  0.00     0    0   

  TurbSC   CHL    CHL.1  
1   20.3  7.15  0.06940  
2    0.0  0.00  0.00000  
3    0.0  0.00  0.00000  
4    0.0  0.00  0.00000  
5    0.0  0.00  0.00000  


In [11]:
# let's get the shapefile for the stationary points collected that day.

station_shp_df = timestamp_match.shp2dataframe(station_shp)

print(station_shp_df.head())

   Site          Date        Time  Sample_Num Zooplankto  Number_of_  \
0   bk1  [2013, 4, 4]  11:23:53am           0        Yes           0   
1   cc1  [2013, 4, 4]  11:48:19am           0        Yes           0   
2  lnca  [2013, 4, 4]  01:35:01pm           0        Yes           1   
3   ca3  [2013, 4, 4]  01:48:55pm           0        Yes           0   
4   hs1  [2013, 4, 4]  04:24:08pm           0        Yes           0   

  Chlorophyl WaterQuali Flow_Meter Flow_Mete2      GPS_Date    GPS_Time  \
0        Yes        Yes    08027.0    08061.2  [2013, 4, 4]  11:23:42am   
1        Yes        Yes    08062.5    08088.8  [2013, 4, 4]  11:48:08am   
2        Yes        Yes     8088.8     8142.8  [2013, 4, 4]  01:34:51pm   
3        Yes        Yes    08144.7    08219.1  [2013, 4, 4]  01:48:45pm   
4        Yes        Yes    08217.9    08227.8  [2013, 4, 4]  04:23:58pm   

                                XY  
0   [-121.796676858, 38.275292895]  
1   [-121.79951459, 38.2594570122]  
2  [-

In [12]:
# combine the GPS_date and GPS time fields to match a timestamp to use as join field
timestamp_match.addCombinedDateTime(station_shp_df, "GPS_Date", "GPS_Time")
print(station_shp_df.head())

   Site          Date        Time  Sample_Num Zooplankto  Number_of_  \
0   bk1  [2013, 4, 4]  11:23:53am           0        Yes           0   
1   cc1  [2013, 4, 4]  11:48:19am           0        Yes           0   
2  lnca  [2013, 4, 4]  01:35:01pm           0        Yes           1   
3   ca3  [2013, 4, 4]  01:48:55pm           0        Yes           0   
4   hs1  [2013, 4, 4]  04:24:08pm           0        Yes           0   

  Chlorophyl WaterQuali Flow_Meter Flow_Mete2      GPS_Date    GPS_Time  \
0        Yes        Yes    08027.0    08061.2  [2013, 4, 4]  11:23:42am   
1        Yes        Yes    08062.5    08088.8  [2013, 4, 4]  11:48:08am   
2        Yes        Yes     8088.8     8142.8  [2013, 4, 4]  01:34:51pm   
3        Yes        Yes    08144.7    08219.1  [2013, 4, 4]  01:48:45pm   
4        Yes        Yes    08217.9    08227.8  [2013, 4, 4]  04:23:58pm   

                                XY           Date_Time  
0   [-121.796676858, 38.275292895] 2013-04-04 11:23:42  
1 